In [6]:
from platform import python_version

print(python_version())


3.7.7


In [123]:
pip install pyspark==2.4.4

You should consider upgrading via the 'D:\WPy64-3771\python-3.7.7.amd64\python.exe -m pip install --upgrade pip' command.


In [124]:
pip install findspark 

You should consider upgrading via the 'D:\WPy64-3771\python-3.7.7.amd64\python.exe -m pip install --upgrade pip' command.


In [105]:
import pandas as pd

In [1]:
import findspark
findspark.init()
import pyspark
import pyspark.sql.functions as f
from pyspark.sql import SparkSession
spark = SparkSession.builder.config("spark.driver.host", "localhost").getOrCreate()


In [2]:
data = [("James","","Smith","36636","M",60000),
        ("Michael","Rose","","40288","M",70000),
        ("Robert","","Williams","42114","",400000),
        ("Maria","Anne","Jones","39192","F",500000),
        ("Jen","Mary","Brown","","F",0)]

columns = ["first_name","middle_name","last_name","dob","gender","salary"]
pysparkDF = spark.createDataFrame(data = data, schema = columns)
pysparkDF.printSchema()
pysparkDF.show(truncate=False)

root
 |-- first_name: string (nullable = true)
 |-- middle_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)

+----------+-----------+---------+-----+------+------+
|first_name|middle_name|last_name|dob  |gender|salary|
+----------+-----------+---------+-----+------+------+
|James     |           |Smith    |36636|M     |60000 |
|Michael   |Rose       |         |40288|M     |70000 |
|Robert    |           |Williams |42114|      |400000|
|Maria     |Anne       |Jones    |39192|F     |500000|
|Jen       |Mary       |Brown    |     |F     |0     |
+----------+-----------+---------+-----+------+------+



In [5]:
df=spark.read.parquet("D:\\source\\202204121920-seller_central_opportunity_explorer_category.parquet")
split_col = f.split(df['parent_ids'], '>')
categoryDf = df.withColumn('CategoryId', split_col.getItem(0))
categoryDf = categoryDf.withColumn('SubCategoryId', split_col.getItem(1))

split_col1 = f.split(df['parent_names'], '>')
categoryDf = categoryDf.withColumn('CategoryName', split_col1.getItem(0))
categoryDf = categoryDf.withColumn('SubCategoryName', split_col1.getItem(1))

categoryDf=categoryDf.withColumnRenamed("category_id","cat_id")
categoryDf=categoryDf.withColumnRenamed("report_date","rep_date")
categoryDf.limit(10).toPandas()
#categoryDf.toPandas().to_csv("categories.csv", header=True)

categoryDf.registerTempTable("ct") 
df10=spark.sql("select * from ct where category_name='Appliances'")
df10.toPandas()
#for field in categoryDf.schema.fields:
 #   print(field.name +" , "+str(field.dataType))
#df1=df.where("parent_ids = ''")
#df1=df1.withColumnRenamed("report_date","report_date_category")


,rep_date,marketplaceid,cat_id,category_name,parent_ids,parent_names,CategoryId,SubCategoryId,CategoryName,SubCategoryName
0,2022-04-12 05:45:00,ATVPDKIKX0DER,2619526011,Appliances,,,,None,,None
1,2022-04-12 05:45:00,ATVPDKIKX0DER,13397451,Appliances,468240,Home Improvement,468240,None,Home Improvement,None


In [38]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

df1=spark.read.parquet("D:\\source\\202204121920-seller_central_opportunity_explorer_niche_summary.parquet")
df1.limit(5).toPandas()

df1=df1.withColumn("TotalSales_360",df1.niche_summary_maximum_units_sold_t360 * df1.niche_summary_avg_price)
df1.registerTempTable("niches") 
df2=df1.join(categoryDf,df1["category_id"] == categoryDf["cat_id"])
df2=df2.withColumn('report_year',f.year(f.to_timestamp('report_date', 'YYYYY-MM-DD')))
df2.limit(10).toPandas()

#for field in df2.schema.fields:
 #   print(field.name +" , "+str(field.dataType))
#df2.toPandas().to_csv("join.csv", header=True)
#windowSpec  = Window.partitionBy("category_name,SubCategoryName").orderBy("total_sales_360 desc")
#df3.toPandas().to_csv("sample_file.csv", header=True)


#df3.withColumn("row_number",row_number().over(windowSpec)).show(truncate=False)
#df3.withColumn("row_number", f.row_number().over(Window.partitionBy("category_name","SubCategoryName").orderBy(desc("total_sales_360"))).show()

#df1.select('category_id').distinct().show()

,report_date,marketplaceid,niche_id,niche_title,reference_asin_image_url,currency,top_search_term_metrics,niche_summary_search_volume_t90,niche_summary_search_volume_growth_t90,niche_summary_search_volume_t360,...,marketplaceid,cat_id,category_name,parent_ids,parent_names,CategoryId,SubCategoryId,CategoryName,SubCategoryName,report_year
0,2022-04-12 05:45:00,ATVPDKIKX0DER,ceef3b04438b73fdb7cc7f84d647e051,sunbeam humidifier filter,https://m.media-amazon.com/images/I/51PPtChJGQ...,USD,sunbeam humidifier filter|hwf62 humidifier fil...,4834,0.070653,11013,...,ATVPDKIKX0DER,2619526011,Appliances,,,,None,,None,2022
1,2022-04-12 05:45:00,ATVPDKIKX0DER,7fe72cbd5e41e782f44ecef89d7a545e,279838,https://m.media-amazon.com/images/I/51gPgOwrcM...,USD,279838 dryer heating element|279838,5345,0.062836,21407,...,ATVPDKIKX0DER,2619526011,Appliances,,,,None,,None,2022
2,2022-04-12 05:45:00,ATVPDKIKX0DER,b92ae9eff10a63d604cbf78e98eeb6f0,humidifier pad,https://m.media-amazon.com/images/I/51GC4gl2EY...,USD,aprilaire|aprilaire 600 replacement filter|apr...,58708,-0.156288,167159,...,ATVPDKIKX0DER,2619526011,Appliances,,,,None,,None,2022
3,2022-04-12 05:45:00,ATVPDKIKX0DER,3c6e61ebbf6b8be0b42b9e461f654b98,flat dryer vent,https://m.media-amazon.com/images/I/31eym3TOyS...,USD,periscope dryer vent|flat dryer vent|dryer ven...,12033,0.126791,46723,...,ATVPDKIKX0DER,2619526011,Appliances,,,,None,,None,2022
4,2022-04-12 05:45:00,ATVPDKIKX0DER,562b5acb9eeddb9f1ab4aee507225e1b,humidifier liquid,https://m.media-amazon.com/images/I/41kREg3zJ-...,USD,menthol oil|menthol oil essential oil|humidifi...,10640,0.047657,36019,...,ATVPDKIKX0DER,2619526011,Appliances,,,,None,,None,2022
5,2022-04-12 05:45:00,ATVPDKIKX0DER,ecba3226111316e9e79a7680c2c7c9ac,dryer filter replacement,https://m.media-amazon.com/images/I/41PL2xupsb...,USD,dryer filter replacement|dryer filter|lint fil...,6370,0.142396,23256,...,ATVPDKIKX0DER,2619526011,Appliances,,,,None,,None,2022
6,2022-04-12 05:45:00,ATVPDKIKX0DER,38cb992f58b18a8b2dabf1d58079814f,lfxs26973s water filter lg,https://m.media-amazon.com/images/I/41-1pBrHAQ...,USD,pureplus water filter|lfxs26973s water filter ...,6745,0.044118,24413,...,ATVPDKIKX0DER,2619526011,Appliances,,,,None,,None,2022
7,2022-04-12 05:45:00,ATVPDKIKX0DER,d58589c683e6762958a06c98e5545e75,dryer vent box,https://m.media-amazon.com/images/I/31YCgVCM3D...,USD,dryer vent box|dryer box|recessed dryer vent b...,16087,0.067202,50995,...,ATVPDKIKX0DER,2619526011,Appliances,,,,None,,None,2022
8,2022-04-12 05:45:00,ATVPDKIKX0DER,491724b8896990061bacd42599ee922d,samsung dryer belt,https://m.media-amazon.com/images/I/515bZeGCfe...,USD,samsung dryer belt|samsung dryer belt replacem...,3845,-0.051554,16336,...,ATVPDKIKX0DER,2619526011,Appliances,,,,None,,None,2022
9,2022-04-12 05:45:00,ATVPDKIKX0DER,bcd628ef393c8f7d28cde54e3d326c73,rf263teaesg water filter,https://m.media-amazon.com/images/I/51QSj2D1Su...,USD,rwf0700a refrigerator water filter|icepure rwf...,3866,0.238309,13223,...,ATVPDKIKX0DER,2619526011,Appliances,,,,None,,None,2022


In [63]:
df11=spark.sql("select distinct report_date,niche_id,niche_title,niche_summary_search_volume_t360,niche_summary_maximum_units_sold_t360,niche_summary_avg_price,TotalSales_360 from niches")
df11.registerTempTable("niches_products") 
df11.toPandas()

,report_date,niche_id,niche_title,niche_summary_search_volume_t360,niche_summary_maximum_units_sold_t360,niche_summary_avg_price,TotalSales_360
0,2022-04-12 05:45:00,3ef64b8358f5579c9a8610acc13cca5d,da29-00003g,16391,6000,36.938872,221633.232
1,2022-04-12 05:45:00,2c43cf170a97c8a05dee3f6432b56993,280187,4573,1500,42.649222,63973.833
2,2022-04-12 05:45:00,53d11220d38084dff1a68c499eb9f508,floral foam cage,25253,6000,20.595263,123571.578
3,2022-04-12 05:45:00,499a67a08732d8b9bfa97731273a4920,craft supplies & materials,228892,8000,28.018340,224146.720
4,2022-04-12 05:45:00,b9bbe9a87e1e87525a0778da973435a5,plasticine,48200,8000,16.110298,128882.384
...,...,...,...,...,...,...,...
37380,2022-04-12 05:45:00,d15082d821dd0301de088deba9b5f8e7,baby computer for 1 year old,75628,6000,39.834885,239009.310
37381,2022-04-12 05:45:00,4fc015b37e6af9b234193f326423ada4,elf ears,497564,80000,10.245969,819677.520
37382,2022-04-12 05:45:00,e7f2bbd453fabaa35e1a1b29491b34cc,gesture rc car,94604,10000,50.481115,504811.150
37383,2022-04-12 05:45:00,e57975156abe8b56ac4f282223ed4654,garden toys,50792,5000,27.359760,136798.800


In [52]:
df11=spark.sql("select niche_id,niche_title,niche_summary_maximum_units_sold_t360 from niches group by  niche_id,niche_title,niche_summary_maximum_units_sold_t360 having count(*)>1")
df11.toPandas()

,niche_id,niche_title,niche_summary_maximum_units_sold_t360
0,6297f5a25b66b45ccbfb9d48e2ab9ddc,whirlpool fridge drawer,1250
1,56e927ee133341d055f3fb4b08568620,lt1000p lg refrigerator water filter,30000
2,63eb2ef6524cb8852e5a8da7a1bafa00,wax vaporizer,8000
3,66abeb7659adee1b08160a6d7e3d7fa3,orbeez,200000
4,91d1ee462b7d85fff33158a13cb34bba,tins,10000
...,...,...,...
14323,3ce918d4d511fce55e35ded1d0ca0092,300 piece puzzles for adults large piece,125000
14324,836315009a7619de46b5912b1d28acf4,rc helicopters,50000
14325,48f818d9918f5a22072277be6b54d6a0,platform swing,3000
14326,68a0605bfc9048d68134a247356e6c56,wooden cars,6000


In [48]:
df11=spark.sql("select a.*,b.*,a.TotalSales_360 from niches a inner join ct b on b.cat_id=a.category_id where a.niche_id='d1dc1d079af157199b108f8f3dd80572'")
df11.toPandas()

,report_date,marketplaceid,niche_id,niche_title,reference_asin_image_url,currency,top_search_term_metrics,niche_summary_search_volume_t90,niche_summary_search_volume_growth_t90,niche_summary_search_volume_t360,...,marketplaceid,cat_id,category_name,parent_ids,parent_names,CategoryId,SubCategoryId,CategoryName,SubCategoryName,TotalSales_360
0,2022-04-12 05:45:00,ATVPDKIKX0DER,d1dc1d079af157199b108f8f3dd80572,lg washer dryer stacking kit,https://m.media-amazon.com/images/I/41np6x19l9...,USD,lg washer dryer stacking kit|lg stacking kit f...,4189,0.021458,15698,...,ATVPDKIKX0DER,2619526011,Appliances,,,,None,,None,84284.145
1,2022-04-12 05:45:00,ATVPDKIKX0DER,d1dc1d079af157199b108f8f3dd80572,lg washer dryer stacking kit,https://m.media-amazon.com/images/I/41np6x19l9...,USD,lg washer dryer stacking kit|lg stacking kit f...,4189,0.021458,15698,...,ATVPDKIKX0DER,3741181,Parts & Accessories,2619526011,Appliances,2619526011,None,Appliances,None,84284.145
2,2022-04-12 05:45:00,ATVPDKIKX0DER,d1dc1d079af157199b108f8f3dd80572,lg washer dryer stacking kit,https://m.media-amazon.com/images/I/41np6x19l9...,USD,lg washer dryer stacking kit|lg stacking kit f...,4189,0.021458,15698,...,ATVPDKIKX0DER,2232336011,Dryer Parts & Accessories,2619526011 > 3741181,Appliances > Parts & Accessories,2619526011,3741181,Appliances,Parts & Accessories,84284.145


In [43]:
df11=spark.sql("select a.*,b.*,a.TotalSales_360 from niches a inner join ct b on b.cat_id=a.category_id where a.niche_id='fef0b2bc1027552316f6c42a6a661cab'")
df11.toPandas()

,report_date,marketplaceid,niche_id,niche_title,reference_asin_image_url,currency,top_search_term_metrics,niche_summary_search_volume_t90,niche_summary_search_volume_growth_t90,niche_summary_search_volume_t360,...,marketplaceid,cat_id,category_name,parent_ids,parent_names,CategoryId,SubCategoryId,CategoryName,SubCategoryName,TotalSales_360
0,2022-04-12 05:45:00,ATVPDKIKX0DER,fef0b2bc1027552316f6c42a6a661cab,wheel cleaner,https://m.media-amazon.com/images/I/31QR8Aqy8L...,USD,wheel cleaner|tire cleaner|rim cleaner|chemica...,108995,0.245885,463222,...,ATVPDKIKX0DER,15690151,Automotive,,,,None,,None,1361409.6
1,2022-04-12 05:45:00,ATVPDKIKX0DER,fef0b2bc1027552316f6c42a6a661cab,wheel cleaner,https://m.media-amazon.com/images/I/31QR8Aqy8L...,USD,wheel cleaner|tire cleaner|rim cleaner|chemica...,108995,0.245885,463222,...,ATVPDKIKX0DER,15718271,Car Care,15690151,Automotive,15690151,None,Automotive,None,1361409.6
2,2022-04-12 05:45:00,ATVPDKIKX0DER,fef0b2bc1027552316f6c42a6a661cab,wheel cleaner,https://m.media-amazon.com/images/I/31QR8Aqy8L...,USD,wheel cleaner|tire cleaner|rim cleaner|chemica...,108995,0.245885,463222,...,ATVPDKIKX0DER,15718641,Tire & Wheel Care,15690151 > 15718271,Automotive > Car Care,15690151,15718271,Automotive,Car Care,1361409.6


In [57]:
#df2.registerTempTable("categories")  

df11=spark.sql("select * from categories where niche_id='fef0b2bc1027552316f6c42a6a661cab'")
df11.toPandas()

,report_date,marketplaceid,niche_id,niche_title,reference_asin_image_url,currency,top_search_term_metrics,niche_summary_search_volume_t90,niche_summary_search_volume_growth_t90,niche_summary_search_volume_t360,...,marketplaceid,cat_id,category_name,parent_ids,parent_names,CategoryId,SubCategoryId,CategoryName,SubCategoryName,report_year
0,2022-04-12 05:45:00,ATVPDKIKX0DER,fef0b2bc1027552316f6c42a6a661cab,wheel cleaner,https://m.media-amazon.com/images/I/31QR8Aqy8L...,USD,wheel cleaner|tire cleaner|rim cleaner|chemica...,108995,0.245885,463222,...,ATVPDKIKX0DER,15690151,Automotive,,,,None,,None,2022
1,2022-04-12 05:45:00,ATVPDKIKX0DER,fef0b2bc1027552316f6c42a6a661cab,wheel cleaner,https://m.media-amazon.com/images/I/31QR8Aqy8L...,USD,wheel cleaner|tire cleaner|rim cleaner|chemica...,108995,0.245885,463222,...,ATVPDKIKX0DER,15718271,Car Care,15690151,Automotive,15690151,None,Automotive,None,2022
2,2022-04-12 05:45:00,ATVPDKIKX0DER,fef0b2bc1027552316f6c42a6a661cab,wheel cleaner,https://m.media-amazon.com/images/I/31QR8Aqy8L...,USD,wheel cleaner|tire cleaner|rim cleaner|chemica...,108995,0.245885,463222,...,ATVPDKIKX0DER,15718641,Tire & Wheel Care,15690151 > 15718271,Automotive > Car Care,15690151,15718271,Automotive,Car Care,2022


In [60]:
df11=spark.sql("select * from niches where niche_id='fef0b2bc1027552316f6c42a6a661cab'")
df11.toPandas()


,report_date,marketplaceid,niche_id,niche_title,reference_asin_image_url,currency,top_search_term_metrics,niche_summary_search_volume_t90,niche_summary_search_volume_growth_t90,niche_summary_search_volume_t360,...,niche_summary_minimum_units_sold_t90,niche_summary_maximum_units_sold_t90,niche_summary_minimum_units_sold_t360,niche_summary_maximum_units_sold_t360,niche_summary_product_count,niche_summary_avg_price,niche_summary_avg_price_t360,category_id,last_updated_time,TotalSales_360
0,2022-04-12 05:45:00,ATVPDKIKX0DER,fef0b2bc1027552316f6c42a6a661cab,wheel cleaner,https://m.media-amazon.com/images/I/31QR8Aqy8L...,USD,wheel cleaner|tire cleaner|rim cleaner|chemica...,108995,0.245885,463222,...,15000,20000,60000,80000,27,17.01762,None,15690151,2022-04-03T00:00:00Z,1361409.6
1,2022-04-12 05:45:00,ATVPDKIKX0DER,fef0b2bc1027552316f6c42a6a661cab,wheel cleaner,https://m.media-amazon.com/images/I/31QR8Aqy8L...,USD,wheel cleaner|tire cleaner|rim cleaner|chemica...,108995,0.245885,463222,...,15000,20000,60000,80000,27,17.01762,None,15718271,2022-04-03T00:00:00Z,1361409.6
2,2022-04-12 05:45:00,ATVPDKIKX0DER,fef0b2bc1027552316f6c42a6a661cab,wheel cleaner,https://m.media-amazon.com/images/I/31QR8Aqy8L...,USD,wheel cleaner|tire cleaner|rim cleaner|chemica...,108995,0.245885,463222,...,15000,20000,60000,80000,27,17.01762,None,15718641,2022-04-03T00:00:00Z,1361409.6


In [32]:
df11=spark.sql("select niche_id,count(*) cnt from categories group by niche_id order by cnt desc")
df11.toPandas()

,niche_id,cnt
0,d1dc1d079af157199b108f8f3dd80572,3
1,fef0b2bc1027552316f6c42a6a661cab,3
2,a7f7d63d99038aa63430912343d32f20,3
3,4a83683e96bbadaf86d1eded3ad76a0c,3
4,25ca3877487fce682a4361a5db2a1a2f,3
...,...,...
37380,e51085f53712400c3e2929170953e6d3,1
37381,86b956f25c0508deb2b2dcb4c1d3de81,1
37382,8d1d80e681a7cc0faae6ca26106451a8,1
37383,9ccc38275e77ecb8de6e1d25e0c4d86e,1


In [8]:


#ignore above code.Done for analysis
df3=spark.sql("select * from (select a.*,ROW_NUMBER() OVER (PARTITION BY category_name,SubCategoryName ORDER BY total_sales_360 DESC) AS rn from (select niche_id,report_year,niche_title,cat_id,category_name,SubCategoryName,parent_ids,round(sum(TotalSales_360),2) as total_sales_360 from categories group by niche_id,report_year,niche_title,cat_id,category_name,SubCategoryName,parent_ids order by cat_id,total_sales_360 desc) a) where rn=1")
df3.toPandas()
 
#df4=df3.filter("SubCategoryName is NOT NULL") 
#df4.toPandas().to_csv("sample_file_drop.csv", header=True)

#df5=df3.where("parent_ids = ''")
#df5.toPandas().to_csv("qno2.csv", header=True)

,niche_id,report_year,niche_title,cat_id,category_name,SubCategoryName,parent_ids,total_sales_360,rn
0,0e61a5b30e170d951021ad70513c0316,2022,baby headphones,21393842011,Hearing Protection Earmuffs,Safety,165797011 > 166863011,1781285.60,1
1,a6d2cf5f128f59c73b10ee6b7c9f0814,2022,stocking stuffers,3761351,Lip Care,Skin Care,11055981 > 11060451,14164366.00,1
2,9592b6c83a55226822b162b37699f24f,2022,frame paint,15709851,Paints & Primers,Paint & Paint Supplies,15690151 > 13591416011,1289989.50,1
3,6afcba65c9937bfd204e36e2c351ea0b,2022,chicken coop,4619362011,Poultry Care,Farm & Ranch,3238155011 > 4619352011,6370485.96,1
4,c6a3c31d6308ea6116a46931c8ae34ed,2022,timing belt kit,15721321,"Belts, Hoses & Pulleys",Replacement Parts,15690151 > 15719731,565721.75,1
...,...,...,...,...,...,...,...,...,...
1768,98698c7c3d4467b8342a7bce22586ae1,2022,couch cover,1063300,Slipcovers,Home Décor Products,1063498 > 1063278,16157819.00,1
1769,210374d4fd6f42670ed7059622e7c413,2022,smart plug,495266,Electrical,None,468240,18110018.40,1
1770,09580c00e2c391d366eba3fe3287e898,2022,flax seed,16310281,"Herbs, Spices & Seasonings",Pantry Staples,16310211 > 18787303011,3928645.20,1
1771,b1a458da2f2736c64f153f2c9848c370,2022,tv speakers for hard of hearing,689637011,Portable Speakers & Docks,Portable Audio & Video,493964 > 172623,729753.32,1


In [18]:
df6=spark.read.parquet("D:\\source\\202204121921-seller_central_opportunity_explorer_niche_search_term_metrics.parquet")
df6.limit(5).toPandas()

#df6.select('report_date').distinct().show()

,report_date,niche_id,niche_search_term_rank,search_term_id,search_term,search_conversion_rate,search_volume_t90,search_volume_qoq,search_volume_yoy,search_volume_t360,...,click_share_t360,search_asin_1,search_asin_title_1,search_asin_image_url_1,search_asin_2,search_asin_title_2,search_asin_image_url_2,search_asin_3,search_asin_title_3,search_asin_image_url_3
0,2022-04-12 05:45:00,ceef3b04438b73fdb7cc7f84d647e051,1,ceef3b04438b73fdb7cc7f84d647e051,sunbeam humidifier filter,0.131220,3376,0.187478,2.365902,6999,...,0.563306,B07S8CNT2J,Strengthmix 2 Pack Filters Compatible with Hol...,https://m.media-amazon.com/images/I/51xNjZ9jV7...,B07HKGJR38,Colorfullife 2 Pack Filters Compatible with Ho...,https://m.media-amazon.com/images/I/51PPtChJGQ...,B07HKKLNS7,Colorfullife 4 Pack Filters Compatible with Ho...,https://m.media-amazon.com/images/I/51rrHQxIea...
1,2022-04-12 05:45:00,ceef3b04438b73fdb7cc7f84d647e051,2,cf7aca817c35a670429a47d8b4bfdbb3,hwf62 humidifier filter replacement,0.315517,1160,1.000000,0.000000,1767,...,0.206533,B07XGPK5HY,Colorfullife 6 Pack Premium Humidifier Filters...,https://m.media-amazon.com/images/I/51IzLubgAP...,B07HKF94H6,Colorfullife 3 Pack Filters Compatible with Ho...,https://m.media-amazon.com/images/I/51qroO4G3i...,B07CV7VNL8,Durabasics 4 Pack Compatible Holmes HWF62 Humi...,https://m.media-amazon.com/images/I/51OIS8N-Nk...
2,2022-04-12 05:45:00,ceef3b04438b73fdb7cc7f84d647e051,3,63d926fc7786a059b41ff489a97900d0,hwf62 humidifier filter,0.259542,262,-0.753296,-0.798616,1803,...,0.191189,B07XGPK5HY,Colorfullife 6 Pack Premium Humidifier Filters...,https://m.media-amazon.com/images/I/51IzLubgAP...,B07CV7VNL8,Durabasics 4 Pack Compatible Holmes HWF62 Humi...,https://m.media-amazon.com/images/I/51OIS8N-Nk...,B072MQ8MX8,Ximoon HWF62 Humidifier Filter A Replacement f...,https://m.media-amazon.com/images/I/51trTSTVE1...
3,2022-04-12 05:45:00,ceef3b04438b73fdb7cc7f84d647e051,4,ac454184b8d73ac137d47f49c2c77d57,sunbeam humidifier filters replacement,0.111111,36,0.200000,-0.982116,444,...,0.038972,B07HKGJR38,Colorfullife 2 Pack Filters Compatible with Ho...,https://m.media-amazon.com/images/I/51PPtChJGQ...,B08CH4SWTT,4 Pack HWF62 Humidifier Filter Replacement for...,https://m.media-amazon.com/images/I/61BsNaMKo7...,B07HKKLNS7,Colorfullife 4 Pack Filters Compatible with Ho...,https://m.media-amazon.com/images/I/51rrHQxIea...
4,2022-04-12 05:45:00,7fe72cbd5e41e782f44ecef89d7a545e,1,0d3dd31554f84b9838fd3db3b1bf0b7c,279838 dryer heating element,0.300211,2848,0.077563,0.126137,10247,...,0.503642,B07YWK935R,279838 Dryer Heating Element 3977767 3392519 T...,https://m.media-amazon.com/images/I/51LSOdEqjr...,B07QTHKXY2,279838 Dryer Heating Element Assembly Replacem...,https://m.media-amazon.com/images/I/51gPgOwrcM...,B07TT21PJ4,279838 Dryer Heating Element 3977767 3392519 D...,https://m.media-amazon.com/images/I/51c+m4hvWR...


In [19]:
df6.registerTempTable("search")  
df6=spark.sql("select * from search")
df6.toPandas()

,report_date,niche_id,niche_search_term_rank,search_term_id,search_term,search_conversion_rate,search_volume_t90,search_volume_qoq,search_volume_yoy,search_volume_t360,...,click_share_t360,search_asin_1,search_asin_title_1,search_asin_image_url_1,search_asin_2,search_asin_title_2,search_asin_image_url_2,search_asin_3,search_asin_title_3,search_asin_image_url_3
0,2022-04-12 05:45:00,ceef3b04438b73fdb7cc7f84d647e051,1,ceef3b04438b73fdb7cc7f84d647e051,sunbeam humidifier filter,0.131220,3376,0.187478,2.365902,6999,...,0.563306,B07S8CNT2J,Strengthmix 2 Pack Filters Compatible with Hol...,https://m.media-amazon.com/images/I/51xNjZ9jV7...,B07HKGJR38,Colorfullife 2 Pack Filters Compatible with Ho...,https://m.media-amazon.com/images/I/51PPtChJGQ...,B07HKKLNS7,Colorfullife 4 Pack Filters Compatible with Ho...,https://m.media-amazon.com/images/I/51rrHQxIea...
1,2022-04-12 05:45:00,ceef3b04438b73fdb7cc7f84d647e051,2,cf7aca817c35a670429a47d8b4bfdbb3,hwf62 humidifier filter replacement,0.315517,1160,1.000000,0.000000,1767,...,0.206533,B07XGPK5HY,Colorfullife 6 Pack Premium Humidifier Filters...,https://m.media-amazon.com/images/I/51IzLubgAP...,B07HKF94H6,Colorfullife 3 Pack Filters Compatible with Ho...,https://m.media-amazon.com/images/I/51qroO4G3i...,B07CV7VNL8,Durabasics 4 Pack Compatible Holmes HWF62 Humi...,https://m.media-amazon.com/images/I/51OIS8N-Nk...
2,2022-04-12 05:45:00,ceef3b04438b73fdb7cc7f84d647e051,3,63d926fc7786a059b41ff489a97900d0,hwf62 humidifier filter,0.259542,262,-0.753296,-0.798616,1803,...,0.191189,B07XGPK5HY,Colorfullife 6 Pack Premium Humidifier Filters...,https://m.media-amazon.com/images/I/51IzLubgAP...,B07CV7VNL8,Durabasics 4 Pack Compatible Holmes HWF62 Humi...,https://m.media-amazon.com/images/I/51OIS8N-Nk...,B072MQ8MX8,Ximoon HWF62 Humidifier Filter A Replacement f...,https://m.media-amazon.com/images/I/51trTSTVE1...
3,2022-04-12 05:45:00,ceef3b04438b73fdb7cc7f84d647e051,4,ac454184b8d73ac137d47f49c2c77d57,sunbeam humidifier filters replacement,0.111111,36,0.200000,-0.982116,444,...,0.038972,B07HKGJR38,Colorfullife 2 Pack Filters Compatible with Ho...,https://m.media-amazon.com/images/I/51PPtChJGQ...,B08CH4SWTT,4 Pack HWF62 Humidifier Filter Replacement for...,https://m.media-amazon.com/images/I/61BsNaMKo7...,B07HKKLNS7,Colorfullife 4 Pack Filters Compatible with Ho...,https://m.media-amazon.com/images/I/51rrHQxIea...
4,2022-04-12 05:45:00,7fe72cbd5e41e782f44ecef89d7a545e,1,0d3dd31554f84b9838fd3db3b1bf0b7c,279838 dryer heating element,0.300211,2848,0.077563,0.126137,10247,...,0.503642,B07YWK935R,279838 Dryer Heating Element 3977767 3392519 T...,https://m.media-amazon.com/images/I/51LSOdEqjr...,B07QTHKXY2,279838 Dryer Heating Element Assembly Replacem...,https://m.media-amazon.com/images/I/51gPgOwrcM...,B07TT21PJ4,279838 Dryer Heating Element 3977767 3392519 D...,https://m.media-amazon.com/images/I/51c+m4hvWR...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504107,2022-04-12 05:45:00,61d0c830f6081cb373c1b837335c85b6,8,56dd4f01e4d3ba255a5fe05f198c8e95,diecast planes,0.020370,540,-0.428571,-0.267300,2853,...,0.031520,B01MFCU7ZZ,Daron American Airlines Delta & United Airline...,https://m.media-amazon.com/images/I/51ef4PbM1U...,B01KW3G6TS,Metal die cast Toy air Plane Set of Military P...,https://m.media-amazon.com/images/I/414KhqHZKu...,B08GF3WTMN,YEIBOBO ! F-16 Fighting Falcon - 1/100 Diecast...,https://m.media-amazon.com/images/I/41Nq2gdiBY...
504108,2022-04-12 05:45:00,61d0c830f6081cb373c1b837335c85b6,9,4ec1c0aaa0509d0f83ef5dbd2d626bf9,matchbox sky busters,0.030060,499,-0.497988,-0.246224,2718,...,0.025599,B09LTJX82Q,DieCast Matchbox Boeing 747-8 Intercontinental...,https://m.media-amazon.com/images/I/517-kgUn+n...,B083HNRGSC,Matchbox Sky Busters Rutan Boomerang 2/13 White,https://m.media-amazon.com/images/I/51XB+gOcyp...,B09CBCZCT8,Matchbox Hondajet [Blue/White] Sky Busters,https://m.media-amazon.com/images/I/51mDsR-ahm...
504109,2022-04-12 05:45:00,61d0c830f6081cb373

In [24]:
#df6.registerTempTable("search")  
df7=spark.sql("select niche_id,sum(search_volume_t360) search_volume_t360,sum(search_volume_yoy) search_volume_yoy,count(*) as cnt from search where niche_id='ceef3b04438b73fdb7cc7f84d647e051' group by niche_id   order by cnt desc")
df7.toPandas()

,niche_id,search_volume_t360,search_volume_yoy,cnt
0,ceef3b04438b73fdb7cc7f84d647e051,11013,0.58517,4


In [29]:
df7=spark.read.parquet("D:\\source\\202204121923-seller_central_opportunity_explorer_niche_asin_metrics.parquet")
df7.limit(5).toPandas()

#df7.registerTempTable("clicks")  

#df7.select('launch_date').distinct().show()

,report_date,niche_id,niche_asin_rank,asin,asin_title,asin_image_url,avg_price,brand,category,launch_date,click_count,click_share_t90,currency,customer_rating,best_sellers_ranking,avg_seller_vendor_count,total_reviews,click_count_t360,click_share_t360,avg_seller_vendor_count_t360
0,2022-04-12 05:45:00,ceef3b04438b73fdb7cc7f84d647e051,1,B07HKGJR38,Colorfullife 2 Pack Filters Compatible with Ho...,https://m.media-amazon.com/images/I/51PPtChJGQ...,15.163877,Colorfullife,Parts & Accessories/Humidifier Parts & Accesso...,2018-09-21,1233,0.255174,USD,4.5,24.7473,4,1349,2380,0.224253,4
1,2022-04-12 05:45:00,ceef3b04438b73fdb7cc7f84d647e051,2,B07HKF94H6,Colorfullife 3 Pack Filters Compatible with Ho...,https://m.media-amazon.com/images/I/51qroO4G3i...,17.630814,Colorfullife,Parts & Accessories/Humidifier Parts & Accesso...,2018-09-21,461,0.095406,USD,4.5,72.1648,1,770,902,0.084990,4
2,2022-04-12 05:45:00,ceef3b04438b73fdb7cc7f84d647e051,3,B07XGPK5HY,Colorfullife 6 Pack Premium Humidifier Filters...,https://m.media-amazon.com/images/I/51IzLubgAP...,23.549366,Colorfullife,Parts & Accessories/Humidifier Parts & Accesso...,2019-09-07,408,0.084437,USD,4.6,196.8462,2,789,1025,0.096580,2
3,2022-04-12 05:45:00,ceef3b04438b73fdb7cc7f84d647e051,4,B07CV7VNL8,Durabasics 4 Pack Compatible Holmes HWF62 Humi...,https://m.media-amazon.com/images/I/51OIS8N-Nk...,21.014224,Durabasics,Parts & Accessories/Humidifier Parts & Accesso...,2018-05-03,377,0.078022,USD,4.6,56.2967,2,605,895,0.084331,2
4,2022-04-12 05:45:00,ceef3b04438b73fdb7cc7f84d647e051,5,B07HKKLNS7,Colorfullife 4 Pack Filters Compatible with Ho...,https://m.media-amazon.com/images/I/51rrHQxIea...,19.738609,Colorfullife,Parts & Accessories/Humidifier Parts & Accesso...,2018-09-21,304,0.062914,USD,4.6,74.5165,3,852,952,0.089701,3


In [64]:
df8=spark.sql("select niche_id,sum(click_share_t360) as click_share_t360,sum(click_count_t360) as click_count_t360,count(*) as cnt from clicks group by niche_id order by niche_id desc") 

df8.registerTempTable("clicks_analysis")
df8.toPandas()

,niche_id,click_share_t360,click_count_t360,cnt
0,ffffc062c196e0d4b975848506ed9f7c,0.962846,575654,36
1,ffff443d7c705b4260cca30c6ee898ce,0.799317,109273,10
2,fffc7521eeeb0ea34f2eed94cf7c5f42,0.908131,26759,18
3,fffb140c831acd1b83d6cd620db7bebd,0.954417,128370,14
4,fff7eefa3324da3e5c936f6bbad83453,0.964399,236328,37
...,...,...,...,...
37373,000e931c1add0cad69a9b340c6ac4f4a,1.000000,5040,13
37374,000e138db5aec8208d5bcd8d9de844b4,1.000000,271458,39
37375,000d07c6aa33e67b6d30d21ec45906d7,0.954698,26174,36
37376,00024b11d378764d5502adc6a30731c5,0.270646,72047,25


In [75]:
df15=spark.sql("select report_date,a.niche_id,niche_title,click_share_t360,click_count_t360,niche_summary_search_volume_t360,niche_summary_maximum_units_sold_t360,TotalSales_360 from clicks_analysis a inner join niches_products b on a.niche_id=b.niche_id") 
df15.toPandas()
df15.registerTempTable("semi")
df15.toPandas()

,report_date,niche_id,niche_title,click_share_t360,click_count_t360,niche_summary_search_volume_t360,niche_summary_maximum_units_sold_t360,TotalSales_360
0,2022-04-12 05:45:00,ffffc062c196e0d4b975848506ed9f7c,mannequin,0.962846,575654,927652,25000,1.718864e+06
1,2022-04-12 05:45:00,ffff443d7c705b4260cca30c6ee898ce,hammock camping chair,0.799317,109273,206706,6000,4.968827e+05
2,2022-04-12 05:45:00,fffc7521eeeb0ea34f2eed94cf7c5f42,alto saxophone mouthpiece,0.908131,26759,32126,5000,3.934912e+05
3,2022-04-12 05:45:00,fffb140c831acd1b83d6cd620db7bebd,aux to bluetooth,0.954417,128370,159545,20000,3.657442e+05
4,2022-04-12 05:45:00,fff7eefa3324da3e5c936f6bbad83453,tpu filament 1.75,0.964399,236328,149417,25000,5.733024e+05
...,...,...,...,...,...,...,...,...
37373,2022-04-12 05:45:00,000e931c1add0cad69a9b340c6ac4f4a,tecumseh starter 37000,1.000000,5040,4545,750,4.652916e+04
37374,2022-04-12 05:45:00,000e138db5aec8208d5bcd8d9de844b4,20x24 picture frame,1.000000,271458,145265,15000,6.854018e+05
37375,2022-04-12 05:45:00,000d07c6aa33e67b6d30d21ec45906d7,mardi gras wreath,0.954698,26174,26454,2500,9.777411e+04
37376,2022-04-12 05:45:00,00024b11d378764d5502adc6a30731c5,mini makeup bag,0.270646,72047,143601,3000,3.375648e+04


In [72]:
df16=spark.read.parquet("D:\\source\\202204121923-seller_central_opportunity_explorer_niche_launch_potential.parquet")
df16.limit(5).toPandas()

#df16.registerTempTable("yoy")  

#df8.select('report_date').distinct().show()

,report_date,niche_id,product_count_current_value,product_count_qoq,product_count_yoy,sponsored_products_percentage_current_value,sponsored_products_percentage_qoq,sponsored_products_percentage_yoy,sponsored_products_percentage_t360_current_value,sponsored_products_percentage_t360_qoq,...,successful_launches_t360_yoy,avg_oos_rate_current_value,avg_oos_rate_qoq,avg_oos_rate_yoy,avg_oos_rate_t360_current_value,avg_oos_rate_t360_qoq,avg_oos_rate_t360_yoy,avg_detail_page_quality_current_value,avg_detail_page_quality_qoq,avg_detail_page_quality_yoy
0,2022-04-12 05:45:00,ceef3b04438b73fdb7cc7f84d647e051,19,24,13,0.947368,0.875000,0.923077,0.904762,0.875000,...,3,0.044534,0.070513,0.118343,0.058608,0.070513,0.118343,91.807018,0.0,0.0
1,2022-04-12 05:45:00,7fe72cbd5e41e782f44ecef89d7a545e,11,11,13,0.909091,0.909091,0.923077,0.900000,0.909091,...,0,0.000000,0.006993,0.106509,0.000000,0.006993,0.106509,95.462121,0.0,0.0
2,2022-04-12 05:45:00,b92ae9eff10a63d604cbf78e98eeb6f0,16,14,15,0.937500,0.928571,0.866667,0.933333,0.928571,...,0,0.024038,0.000000,0.076923,0.025641,0.000000,0.076923,96.093750,0.0,0.0
3,2022-04-12 05:45:00,3c6e61ebbf6b8be0b42b9e461f654b98,20,17,18,0.450000,0.470588,0.555556,0.450000,0.470588,...,0,0.169231,0.244344,0.260684,0.203846,0.244344,0.260684,80.818750,0.0,0.0
4,2022-04-12 05:45:00,562b5acb9eeddb9f1ab4aee507225e1b,6,5,15,0.666667,0.800000,0.733333,0.875000,0.800000,...,1,0.012821,0.015385,0.092308,0.009615,0.015385,0.092308,85.625000,0.0,0.0


In [78]:
df17=spark.sql("select niche_id,sum(product_count_yoy) as product_count_yoy from yoy group by niche_id")
df17.registerTempTable("final")
df17.toPandas()


,niche_id,product_count_yoy
0,d1dc1d079af157199b108f8f3dd80572,8
1,a5e8e919b958fc7de834f58dc20b6293,52
2,cbd2f2678b09623244b978cf2a54094e,11
3,51639af6c5c2a0bb1970bfe6edad9980,46
4,630094ec768e2ba4cfb633f583bdf8d4,61
...,...,...
37380,86b956f25c0508deb2b2dcb4c1d3de81,24
37381,634b7ef79d258529113e977ff66ec4ff,47
37382,8d1d80e681a7cc0faae6ca26106451a8,17
37383,5b2ee4544d539f0d48c051bcb937c591,73


In [107]:
#df18=spark.sql("select *  from semi as a inner join final b on a.niche_id=b.niche_id")
#df18.registerTempTable("final_anal")
#df18.toPandas() 
df18.toPandas().to_csv("", header=True)

PermissionError: [Errno 13] Permission denied: 'D:\\WPy64-3771\\notebooks\\qno3.xlsx'

In [115]:
pdDF  = pd.read_csv('D:\\WPy64-3771\\notebooks\\qno3.csv')


In [124]:
from pyspark.sql.types import *

mySchema = StructType([ StructField("report_date",StringType(), True)\
                       ,StructField("niche_id", StringType(), True)\
                       ,StructField("niche_title", StringType(), True)\
                       ,StructField("click_share_t360", FloatType(), True)\
                       ,StructField("click_count_t360", IntegerType(), True)\
                       ,StructField("niche_summary_search_volume_t360", IntegerType(), True)\
                       ,StructField("niche_summary_maximum_units_sold_t360", IntegerType(), True)\
                       ,StructField("product_count_yoy", IntegerType(), True)\
                       ,StructField("TotalSales_360", FloatType(), True)\
                       ])

In [125]:
df = spark.createDataFrame(pdDF,schema=mySchema)

In [126]:
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
report_date,37378,2022.0,0.0,2022,2022
niche_id,37378,None,None,0001bd723547963e9bc72f3b8a976e50,ffffc062c196e0d4b975848506ed9f7c
niche_title,37378,9.254981466666667E7,1.1637353371962231E8,#9 envelopes,zwilling scissors
click_share_t360,37378,0.9472938174637693,0.12602225727523542,0.011252813,1.0
click_count_t360,37378,304852.82717106317,922105.6317794463,150,34595862
niche_summary_search_volume_t360,37378,281456.32283696294,846181.1886551279,2434,25196033
niche_summary_maximum_units_sold_t360,37378,36216.305045748835,98080.88762840605,100,1000000
product_count_yoy,37378,33.081652308844774,31.504104857522663,1,785
TotalSales_360,37378,1123105.273797465,5213333.563756497,692.3,3.19665024E8


In [127]:
import six
for i in df.columns:
    if not( isinstance(df.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to Sales for ", i, df.stat.corr('TotalSales_360',i))

Correlation to Sales for  click_share_t360 0.04801700444464939
Correlation to Sales for  click_count_t360 0.6106991824499717
Correlation to Sales for  niche_summary_search_volume_t360 0.697499141592546
Correlation to Sales for  niche_summary_maximum_units_sold_t360 0.4999795785990298
Correlation to Sales for  product_count_yoy 0.02857406905565327
Correlation to Sales for  TotalSales_360 1.0


In [128]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['click_share_t360', 'click_count_t360', 'niche_summary_search_volume_t360', 'niche_summary_maximum_units_sold_t360', 'product_count_yoy', 'TotalSales_360'], outputCol = 'features')
vsales_df = vectorAssembler.transform(df)
vsales_df = vsales_df.select(['features', 'TotalSales_360'])
vsales_df.show(3)

+--------------------+--------------+
|            features|TotalSales_360|
+--------------------+--------------+
|[0.96284627914428...|     1718863.5|
|[0.79931676387786...|      496882.7|
|[0.90813142061233...|      393491.2|
+--------------------+--------------+
only showing top 3 rows



In [129]:
splits = vsales_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]


In [132]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='TotalSales_360', maxIter=10, regParam=0.8, elasticNetParam=0.3)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [140101.5278928627,-0.09236206883021272,0.29719690086624934,-1.0866173336037706,-629.6004379577837,0.9822428347195576]
Intercept: -107918.12332885827


In [131]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 109026.308363
r2: 0.999459


In [134]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","TotalSales_360","features").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="TotalSales_360",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+-------------------+--------------+--------------------+
|         prediction|TotalSales_360|            features|
+-------------------+--------------+--------------------+
|-55028.648263344425|       14953.4|[0.02013335376977...|
|-20520.987595406375|       25334.5|[0.05005326122045...|
| 2046.5654065376875|       57871.8|[0.13958255946636...|
|  352942.7061732046|      415286.7|[0.17135486006736...|
| -383.9179513831041|       47107.5|[0.21776698529720...|
+-------------------+--------------+--------------------+
only showing top 5 rows

R Squared (R2) on test data = 0.999569


In [135]:
test_result = lr_model.evaluate(test_df)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 130076


In [142]:
predictions = lr_model.transform(test_df)
df20=predictions.select("prediction","TotalSales_360","features")

In [138]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol = 'features', labelCol = 'TotalSales_360', maxIter=10)
gbt_model = gbt.fit(train_df)
gbt_predictions = gbt_model.transform(test_df)
gbt_predictions.select('prediction', 'TotalSales_360', 'features').show(5)

+-----------------+--------------+--------------------+
|       prediction|TotalSales_360|            features|
+-----------------+--------------+--------------------+
|61604.42062295016|       14953.4|[0.02013335376977...|
| 64653.3171855542|       25334.5|[0.05005326122045...|
|74679.08618366608|       57871.8|[0.13958255946636...|
|  378325.52001826|      415286.7|[0.17135486006736...|
|61604.42062295016|       47107.5|[0.21776698529720...|
+-----------------+--------------+--------------------+
only showing top 5 rows



In [140]:
gbt_evaluator = RegressionEvaluator(
    labelCol="TotalSales_360", predictionCol="prediction", metricName="rmse")
rmse = gbt_evaluator.evaluate(gbt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 4.00011e+06


In [141]:
from pyspark.ml.regression import DecisionTreeRegressor
dt = DecisionTreeRegressor(featuresCol ='features', labelCol = 'TotalSales_360')
dt_model = dt.fit(train_df)
dt_predictions = dt_model.transform(test_df)
dt_evaluator = RegressionEvaluator(
    labelCol="TotalSales_360", predictionCol="prediction", metricName="rmse")
rmse = dt_evaluator.evaluate(dt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 3.78241e+06


In [144]:
df20.toPandas().to_csv("D:\\WPy64-3771\\notebooks\\prediction.csv", header=True)

In [145]:
predictionDf  = pd.read_csv('D:\\WPy64-3771\\notebooks\\prediction.csv')

In [146]:
predictionDfspark = spark.createDataFrame(predictionDf)

In [147]:
predictionDfspark.registerTempTable("predictiontable")

In [150]:
#df18.registerTempTable("final_anal")
predictionDfspark.toPandas()

,prediction_next_year,TotalSales_360,click_share_t360,click_count_t360,niche_summary_search_volume_t360,niche_summary_maximum_units_sold_t360,product_count_yoy
0,-5.502865e+04,14953.4,0.020133,1546,134068,500,6
1,-2.052099e+04,25334.5,0.050053,7377,193898,750,1
2,2.046565e+03,57871.8,0.139583,24376,152086,4000,8
3,3.529427e+05,415286.7,0.171355,30684,321505,50000,15
4,-3.839180e+02,47107.5,0.217767,45441,145808,2500,9
...,...,...,...,...,...,...,...
11237,2.166030e+07,22118758.0,1.000000,8284980,5929202,1000000,13
11238,1.977050e+07,19931714.0,1.000000,8637239,6942056,1000000,29
11239,1.850935e+07,18438812.0,1.000000,10681866,8321061,1000000,54
11240,1.221524e+07,13429897.0,1.000000,14260963,4006423,800000,20


In [156]:
#df20=spark.sql("select a.niche_title,b.totalsales_360,b.prediction_next_year from final_anal a inner join predictiontable b on a.niche_summary_search_volume_t360=b.niche_summary_search_volume_t360 and a.niche_summary_maximum_units_sold_t360=b.niche_summary_maximum_units_sold_t360 and a.product_count_yoy=b.product_count_yoy")  
df20.toPandas().to_csv("FinalPredciton", header=True)

In [154]:
df20=spark.sql("select * from final_anal")  
df20.toPandas()

#df20 = df20.drop("niche_id")